# <center> Topic 10. Gradient Boosting

# 1. Введение и история бустинга
Почти все в области машинного обучения слышали о градиентном бустинге. Многие исследователи данных включают этот алгоритм в свой инструментарий data scientist из-за хороших результатов, которые он дает по любой заданной (неизвестной) проблеме.

Кроме того, XGBoost часто является стандартным рецептом для [победы](https://github.com/dmlc/xgboost/blob/master/demo/README.md#usecases) [конкурсы мл](http://blog.kaggle.com/tag/xgboost/). он настолько популярен, что идея укладки XGBoosts стала мемом. Кроме того, бустинг является важным компонентом в [многих рекомендательных системах](https://en.wikipedia.org/wiki/Learning_to_rank#Practical_usage_by_search_engines); иногда это даже считается [брендом](https://yandex.com/company/technologies/matrixnet/).
Давайте посмотрим на историю и развитие бустинга.

Бустинг родился из [вопроса:](http://www.cis.upenn.edu/~mkearns/papers/boostnote.pdf) можно ли получить одну сильную модель из большого количества относительно слабых и простых моделей? Говоря "слабые модели", мы имеем в виду не простые базовые модели, такие как деревья решений, а модели с низкой точностью исполнения, где плохое немного лучше, чем случайное.

[Положительный математический ответ](http://www.cs.princeton.edu/~schapire/papers/strengthofweak.pdf) на этот вопрос был определен, но потребовалось несколько лет, чтобы разработать полностью функционирующие алгоритмы, основанные на этом решении, например AdaBoost. Эти алгоритмы используют жадный подход: во-первых, они строят линейную комбинацию простых моделей (базовых алгоритмов) путем повторного взвешивания входных данных. Затем модель (обычно дерево решений) строится на основе ранее неверно предсказанных объектов, которые теперь имеют больший вес.

<spoiler title= "подробнее об AdaBoost">
Многие курсы машинного обучения изучают AdaBoost - предок GBM (Gradient Boosting Machine). Однако с тех пор, как AdaBoost слился с GBM, стало очевидно, что AdaBoost - это всего лишь частный вариант GBM.
    
Сам алгоритм имеет очень четкую визуальную интерпретацию и интуицию для определения Весов. Давайте рассмотрим следующую задачу классификации игрушек, в которой мы будем разбивать данные между деревьями глубины 1 (также известными как "пни" - "stumps") на каждой итерации AdaBoost. Для первых двух итераций мы имеем следующую картину:
<img src='https://habrastorage.org/web/d28/78f/7ba/d2878f7bad0340fc8002e5ba6d0879a5.jpg' width=70%>

Размер точки соответствует ее весу, который был присвоен за неправильное предсказание. На каждой итерации мы видим, что эти веса растут-пни не могут справиться с этой проблемой. Хотя, если мы возьмем взвешенное голосование за пни, то получим правильную классификацию:
<img src='https://habrastorage.org/web/b2b/029/d89/b2b029d898f64bbbb158e15d29595969.png' width=70%>
Псевдокод:
- Инициализация весов выборки $\Large w_i^{(0)} = \frac{1}{l}, i = 1, \dots, l$.
- Для всех $t = 1, \dots, T$
    * Train base algo $\Large b_t$, let $\epsilon_t$ be it's training error.
    * $\Large \alpha_t = \frac{1}{2}ln\frac{1 - \epsilon_t}{\epsilon_t}$.
    * Обновление весов примера: $\Large w_i^{(t)} = w_i^{(t-1)} e^{-\alpha_t y_i b_t(x_i)}, i = 1, \dots, l$.
    * Нормализация весов примера: $\Large w_0^{(t)} = \sum_{j = 1}^k w_j^{(t)}, w_i^{(t)} = \frac{w_i^{(t)}}{w_0^{(t)}}, i = 1, \dots, l$.
- Return $\sum_t^{T}\alpha_tb_t$

[Здесь](https://www.youtube.com/watch?v=k4G2VCuOMMg) является более детальным примером AdaBoost, где, как мы повторяем, мы можем видеть увеличение веса, особенно на границе между классами.

AdaBoost работает хорошо, но [отсутствие](https://www.cs.princeton.edu/courses/archive/spring07/cos424/papers/boosting-survey.pdf) объяснения того, почему алгоритм успешен, зашило семена сомнения. Некоторые считали его супер-алгоритмом, серебряной пулей, но другие были скептичны и считали, что Адабуст просто слишком подходит.

Проблема перенасыщения действительно существовала, особенно когда данные имели сильные выбросы. Поэтому в таких типах проблем Адабуст был нестабилен. К счастью, несколько профессоров статистического факультета Стэнфорда, создавших лассо, эластичную сеть и случайный лес, начали исследовать этот алгоритм. В 1999 году Джером Фридман выступил с обобщением разработки алгоритмов бустинга-Gradient Boosting (Machine), также известного как GBM. С помощью этой работы Фридман создал статистическую основу для многих алгоритмов, обеспечивающих общий подход бустинга для оптимизации в функциональном пространстве.

CART, bootstrap, и множество других алгоритмов, возникли из Департамента статистики Стэнфордского университета. Тем самым кафедра закрепила их имена в будущих учебниках. Эти алгоритмы очень практичны, и некоторые недавние работы еще не получили широкого распространения. Например, проверьте [glinternet](https://arxiv.org/abs/1308.2719).

Видеозаписи Фридмана доступны не так уж много. Хотя, есть очень интересное [интервью](https://www.youtube.com/watch?v=8hupHmBVvb0) с ним о создании карт и о том, как они решали задачи статистики (которая сегодня похожа на data analysis и data science) более 40 лет назад.

Существует также большая [лекция](https://www.youtube.com/watch?v=zBk3PK3g-Fc) от Hastie, ретроспектива анализа данных от одного из создателей методов, которые мы используем каждый день.

В целом произошел переход от инженерно-алгоритмических исследований к полноценному подходу к построению и изучению алгоритмов. С математической точки зрения это не большое изменение - мы все еще добавляем (или усиливаем) слабые алгоритмы и расширяем наш ансамбль с постепенным улучшением для тех частей данных, где модель была неточной. Но на этот раз следующая простая модель не просто строится на повторно взвешенных объектах, а улучшает свою аппроксимацию градиента общей целевой функции. Эта концепция значительно открывает наши алгоритмы для воображения и расширения.

<img src="https://habrastorage.org/webt/h2/v4/k9/h2v4k9r-4yn4jwvwz99fbss4ghi.png" />

## История GBM

Потребовалось более 10 лет после внедрения GBM, чтобы он стал неотъемлемой частью инструментария data science toolbox.
GBM был расширен для применения к различным статистическим задачам: GLMboost и GAMboost для усиления уже существующих моделей GAM, CoxBoost для кривых выживания и RankBoost и LambdaMART для ранжирования.
Многие реализации GBM также появились под разными названиями и на разных платформах: Stochastic GBM, GBDT (Gradient Boosted Decision Trees), GBRT (Gradient Boosted Regression Trees), MART (Multiple Additive Regression Trees) и другие. Кроме того, сообщество мл было очень сегментированным и диссоциированным, что затрудняло отслеживание того, насколько широко распространился бустинг.


В то же время бустинг активно использовался в поисковом ранжировании. Эта проблема была переписана в терминах функции потерь, которая штрафует ошибки в порядке вывода, поэтому стало удобно просто вставить ее в GBM. AltaVista была одной из первых компаний, которые ввели повышение рейтинга. Вскоре эти идеи распространились на Yahoo, Yandex, Bing и т. д. Как только это произошло, бустинг стал одним из основных алгоритмов, который использовался не только в исследованиях, но и в основных технологиях в промышленности.


<img src='https://habrastorage.org/web/48a/ea4/fff/48aea4fffdbe4e5f9205ba81110e6061.jpg' align='right' width=30%>

Соревнования мл, особенно Kaggle, сыграли важную роль в популяризации форсирования. Теперь у исследователей была общая платформа, где они могли соревноваться в различных задачах науки о данных с большим количеством участников со всего мира. С помощью Kaggle можно было бы тестировать новые алгоритмы на реальных данных, давая алгоритмам возможность "сиять" и предоставлять полную информацию при обмене результатами работы модели между наборами данных о конкуренции. Это именно то, что произошло с повышением, когда он был использован в [Kaggle](http://blog.kaggle.com/2011/12/21/score-xavier-conort-on-coming-second-in-give-me-some-credit/) (проверьте интервью с победителями Kaggle, начиная с 2011 года, которые в основном использовали бустинг). The [XGBoost](https://github.com/dmlc/xgboost) библиотека быстро завоевала популярность после своего появления. XGBoost-это не новый, уникальный алгоритм; это просто чрезвычайно эффективная реализация классического GBM с дополнительными эвристиками.


Этот алгоритм прошел очень типичный путь для современных алгоритмов ML: математические задачи и алгоритмические ремесла к успешному практическому применению и массовому внедрению спустя годы после его первого появления.

# 2. Алгоритм GBM
### ML постановка задачи

Мы собираемся решить проблему аппроксимации функций в общей контролируемой учебной обстановке. У нас есть набор признаков $ \large x $ и целевые переменные $\large y, \large \left\{ (x_i, y_i) \right\}_{i=1, \ldots,n}$, которые мы используем для восстановления зависимости $\large y = f(x) $. Мы восстанавливаем зависимость, аппроксимируя $ \large \hat{f}(x) $ и понимая, какая аппроксимация лучше, когда мы используем функцию потерь $ \large L(y, f) $, которую мы хотим минимизировать: $ \large y \approx \hat{f}(x), \large \hat{f}(x) = \underset{f(x)} {\arg\min} \ L(y, f(x)) $.

<img src='../../img/topic10_help_with_func.png'  align='center'>

В этот момент, мы не делаем никаких предположений относительно типа зависимости $ \large f(x) $, модели нашего приближения $ \large \hat{f}(x) $, или распределение целевой переменной $ \large y $. Мы только ожидаем, что функция $ \large L(y,f) $ будет дифференцируемой. Наш подход очень общий: мы определяем $ \large \hat {f}(x) $ за счет минимизации потерь:  
$$ \large  \hat{f}(x) = \underset{f(x)}{\arg\min} \ \mathbb {E} _{x,y}[L(y,f(x))]  $$

К сожалению, количество функций $ \large f(x) $ не просто велико, но его функциональное пространство бесконечномерно. Именно поэтому для нас допустимо ограничить пространство поиска некоторым семейством функций $ \large f(x, \theta), \theta \in \mathbb{R}^d $. Это значительно упрощает задачу, потому что теперь у нас есть разрешимая оптимизация значений параметров:
$ \large \hat{f}(x) = f(x, \hat{\theta}),$
$$\large \hat{\theta} = \underset{\theta}{\arg\min} \ \mathbb {E} _{x,y}[L(y,f(x,\theta))] $$

Простые аналитические решения для нахождения оптимальных параметров $ \large \hat{\theta} $ часто не существуют, поэтому параметры обычно аппроксимируются итеративно. Для начала запишем эмпирическую функцию потерь $ \large L_{\theta}(\hat{\theta}) $ - это позволит нам оценить наши параметры, используя наши данные. Кроме того, давайте выпишем наше приближение $ \large \hat{\theta} $ для числа из $ \large M $ итераций в виде суммы:  
$ \large \hat{\theta} = \sum_{i = 1}^M \hat{\theta_i}, \\
\large L_{\theta}(\hat{\theta}) =  \sum_{i = 1}^N L(y_i,f(x_i, \hat{\theta}))$  

Тогда единственное, что остается, - это найти подходящий итерационный алгоритм для минимизации $\large L_{\theta}(\hat{\theta})$. Градиентный спуск - самый простой и часто используемый вариант. Мы определяем градиент как $\large \nabla L_{\theta}(\hat{\theta})$ и добавьте наши итеративные оценки $\large \hat{\theta_i}$ к нему (поскольку мы минимизируем потери, мы добавляем знак минус). Наш последний шаг - это инициализация нашего первого приближения $\large \hat{\theta_0}$ и выберите количество итераций $\large M$. Давайте рассмотрим шаги для этого неэффективного и наивного алгоритма аппроксимации $\large \hat{\theta}$:

1. Определите начальное приближение параметров $\large \hat{\theta} = \hat{\theta_0}$
2. Для каждой итерации $\large t = 1, \dots, M$ повторять шаги 3-7:
1. Вычислите градиент функции потерь $\large \nabla L_{\theta}(\hat{\theta})$ для текущего приближения $\large \hat{\theta}$
$\large \nabla L_{\theta}(\hat{\theta}) = \left[\frac{\partial L(y, f(x, \theta))}{\partial \theta}\right]_{\theta = \hat{\theta}}$
2. Установите текущее итерационное приближение $\large \hat{\theta_t}$ на основе рассчитанного градиента
$\large \hat{\theta_t} \leftarrow −\nabla L_{\theta}(\hat{\theta})$
3. Обновите аппроксимацию параметров $\large \hat{\theta}$:
$\large \hat{\theta} \leftarrow \hat{\theta} + \hat{\theta_t} = \sum_{i = 0}^t \hat{\theta_i} $
3. Сохраните результат аппроксимации $\large \hat{\theta}$:
$\large \hat{\theta} = \sum_{i = 0}^M \hat{\theta_i} $
4. Использовать функцию, которая была найдена $\large \hat{f}(x) = f(x, \hat{\theta})$

<img src='https://habrastorage.org/web/2b5/5d6/90d/2b55d690d99e4ec0976b360aae6ce4df.jpg'   align='center'>

### Функциональный градиентный спуск

Давайте представим на секунду, что мы можем выполнить оптимизацию в функциональном пространстве и итеративно искать аппроксимации $\large \hat{f}(x)$ как и сами функции. Мы выразим наше приближение как сумму постепенных улучшений, каждое из которых является функцией. Для удобства мы сразу же начнем с суммы из начального приближения$\large \hat{f_0}(x)$:
$$\large \hat{f}(x) = \sum_{i = 0}^M \hat{f_i}(x)$$

Пока ничего не произошло, мы только решили, что будем искать наше приближение $\large \hat{f}(x)$ не как большая модель с большим количеством параметров (например, нейронная сеть), а как сумма функций, делая вид, что мы движемся в функциональном пространстве.

Чтобы выполнить эту задачу, нам нужно ограничить наш поиск некоторым семейством функций $\large \hat{f}(x) = h(x, \theta)$. Здесь есть несколько проблем - во-первых, сумма моделей может быть более сложной, чем любая модель из этого семейства; во-вторых, общая цель все еще находится в функциональном пространстве. Отметим, что на каждом этапе нам нужно будет подбирать оптимальный коэффициент $\large \rho \in \mathbb{R}$. Для шага $\large t$, задача заключается в следующем:
$$\large \hat{f}(x) = \sum_{i = 0}^{t-1} \hat{f_i}(x), \\
\large (\rho_t,\theta_t) = \underset{\rho,\theta}{\arg\min} \ \mathbb {E} _{x,y}[L(y,\hat{f}(x) +  \rho \cdot h(x, \theta))], \\
\large \hat{f_t}(x) = \rho_t \cdot h(x, \theta_t)$$

Вот тут-то и происходит волшебство. Мы определили все наши цели в общих чертах, как будто мы могли бы обучить любую модель $\large h(x, \theta)$ для любого типа функций потери $\large L(y, f(x, \theta))$. На практике это крайне сложно, но, к счастью, есть простой способ решить эту задачу.

Зная выражение градиента функции потерь, мы можем вычислить его значение по нашим данным. Итак, давайте обучим модели таким образом, чтобы наши прогнозы были более коррелированы с этим градиентом (со знаком минус). Другими словами, мы будем использовать метод наименьших квадратов для коррекции предсказаний с помощью этих остатков. Для задач классификации, регрессии и ранжирования мы сведем к минимуму квадратную разницу между псевдо-невязками $\large r$ и наши предсказания. Для шага $\large t$, последняя задача выглядит следующим образом:
$$ \large \hat{f}(x) = \sum_{i = 0}^{t-1} \hat{f_i}(x), \\
\large r_{it} = -\left[\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\right]_{f(x)=\hat{f}(x)}, \quad \mbox{for } i=1,\ldots,n ,\\
\large \theta_t = \underset{\theta}{\arg\min} \ \sum_{i = 1}^{n} (r_{it} - h(x_i, \theta))^2, \\
\large \rho_t = \underset{\rho}{\arg\min} \ \sum_{i = 1}^{n} L(y_i, \hat{f}(x_i) + \rho \cdot h(x_i, \theta_t))$$

<img src='../../img/topic10_regression_for_everybody.jpg'   align='center' width=60%>

### Классический алгоритм GBM Фридмана

Теперь мы можем определить классический алгоритм GBM, предложенный Джеромом Фридманом в 1999 году. Это алгоритм с учителем, который имеет следующие компоненты:

- набор данных  $\large \left\{ (x_i, y_i) \right\}_{i=1, \ldots,n}$;
- число итераций $\large M$;
- выбор функции потери $\large L(y, f)$ с определенным градиентом;
- выбор семейства функций базовых алгоритмов $\large h(x, \theta)$ с помощью процедуры обучения;
- дополнительные гиперпараметры $\large h(x, \theta)$ (например, в деревьях принятия решений глубина дерева);

Осталось только начальное приближение $\large f_0(x)$. Для простоты, для начального приближения, постоянная величина $\large \gamma$ используется. Постоянное значение, а также оптимальный коэффициент $\large \rho $, идентифицируются с помощью бинарного поиска или другого алгоритма поиска строк по исходной функции потерь (не градиент). Итак, у нас есть наш алгоритм GBM, описанный следующим образом:

1. Инициализировать GBM с постоянным значением $\large \hat{f}(x) = \hat{f}_0, \hat{f}_0 = \gamma,  \gamma \in \mathbb{R}$
$\large \hat{f}_0 = \underset{\gamma}{\arg\min} \ \sum_{i = 1}^{n} L(y_i, \gamma)$
2. Для каждой итерации $\large t = 1, \dots, M$, повторить:
1. Вычисление псевдо-невязок $\large r_t$
$\large r_{it} = -\left[\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\right]_{f(x)=\hat{f}(x)}, \quad \mbox{for } i=1,\ldots,n$
2. Построить новый базовый алгоритм $\large h_t(x)$ как регрессия по псевдо-остаткам $\large \left\{ (x_i, r_{it}) \right\}_{i=1, \ldots,n}$
3. Найти оптимальный коэффициент $\large \rho_t $ на $\large h_t(x)$ относительно функции начальной потери
$\large \rho_t = \underset{\rho}{\arg\min} \ \sum_{i = 1}^{n} L(y_i, \hat{f}(x_i) +  \rho \cdot h(x_i, \theta))$
4. Сохранить $\large \hat{f_t}(x) = \rho_t \cdot h_t(x)$
5. Обновление текущего приближения: $\large \hat{f}(x)$
$\large \hat{f}(x) \leftarrow \hat{f}(x) + \hat{f_t}(x) = \sum_{i = 0}^{t} \hat{f_i}(x)$
3. Составить окончательную модель GBM: $\large \hat{f}(x)$
$\large \hat{f}(x) = \sum_{i = 0}^M \hat{f_i}(x) $
4. Покорить Kaggle и остальной мир

### Пошаговый пример: как работает GBM

Давайте рассмотрим пример того, как работает GBM. В этом игрушечном примере мы восстановим функцию с шумом $\large y = cos(x) + \epsilon, \epsilon \sim \mathcal{N}(0, \frac{1}{5}), x \in [-5,5]$.

<img src='https://habrastorage.org/web/9fe/04d/7ba/9fe04d7ba5a645d49fc6aa3e875c8c41.jpg'   align='center'>

Это регрессионная задача с реальной целью, поэтому мы будем использовать функцию потери среднеквадратичной ошибки. Мы сгенерируем 300 пар наблюдений и аппроксимируем их деревьями решений глубины 2. Давайте соберем все, что нам нужно, чтобы использовать GBM:
- Игрушечные данные $\large \left\{ (x_i, y_i) \right\}_{i=1, \ldots,300}$ ✓
- Число итераций $\large M = 3$ ✓;
- Среднеквадратичная функция потерь $\large L(y, f) = (y-f)^2$ ✓
- Градиент из $\large L(y, f) = L_2$ потери - это просто остатки $\large r = (y - f)$ ✓;
- Деревья решений как базовые алгоритмы $\large h(x)$ ✓;
- Используются для отбора признаков дерева решений: деревья глубина равна 2 ✓;

Для среднеквадратичной ошибки обе инициализации $\large \gamma$ и коэффициенты $\large \rho_t$ все очень просто. Мы будем инициализировать GBM со средним значением $\large \gamma = \frac{1}{n} \cdot \sum_{i = 1}^n y_i$, и установим все коэффициенты$\large \rho_t$ к 1.

Мы запустим GBM и нарисуем два типа графиков: текущее приближение $\large \hat{f}(x)$ (синий график) и каждое дерево $\large \hat{f_t}(x)$ построен на его псевдо-невязках (зеленый график). Номер графа соответствует номеру итерации:

<img src='https://habrastorage.org/web/edb/328/98a/edb32898ad014d8d95782759d11f63fb.png'   align='center'>

Ко второй итерации наши деревья восстановили основную форму функции. Однако на первой итерации мы видим, что алгоритм построил только "левую ветвь" функции ($\large x \in [-5, -4]$). Это было связано с тем, что наши деревья просто не имели достаточной глубины, чтобы построить симметричную ветвь сразу, и она фокусировалась на левой ветви с большей ошибкой. Поэтому правая ветвь появилась только после второй итерации.

Остальная часть процесса идет, как и ожидалось - на каждом шаге наши псевдо-невязки уменьшались, и GBM аппроксимировала исходную функцию все лучше и лучше с каждой итерацией. Однако при построении деревья не могут аппроксимировать непрерывную функцию, что означает, что GBM не идеален в этом примере. Чтобы играть с приближениями функций GBM, вы можете использовать потрясающую интерактивную демонстрацию в этом блоге под названием [Brilliantly wrong](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html):

<img src='https://habrastorage.org/web/779/3e0/e66/7793e0e66b7d4871b6391a94cd5d4cf2.jpg'   align='center'>

# 3. Функция потерь

Если мы хотим решить проблему классификации вместо регрессии, что изменится? Нам нужно только выбрать подходящую функцию потерь $\large L(y, f)$. Это самый важный, высокоуровневый момент, который точно определяет, как мы будем оптимизировать и какие характеристики мы можем ожидать в конечной модели.

Как правило, нам самим это изобретать не нужно – исследователи уже сделали это за нас. Сегодня мы рассмотрим функции потерь для двух наиболее распространенных целей: регрессии $\large y \in \mathbb{R}$ и бинарная классификация $\large y \in \left\{-1, 1\right\}$. 

### Регрессионные функции потерь

Давайте начнем с регрессионной задачи для $\large y \in \mathbb{R}$. Для того чтобы выбрать соответствующую функцию потерь, нам необходимо рассмотреть, какое из свойств условного распределения $\large (y|x)$ мы хотим его восстановить. Наиболее распространенными вариантами являются:

- $\large L(y, f) = (y - f)^2$ a.k.a. $\large L_2$ потеря или потеря по Гауссу. Это классическое условное среднее, которое является самым простым и распространенным случаем. Если у нас нет никакой дополнительной информации или требований к надежности модели, мы можем использовать гауссову потерю.
- $\large L(y, f) = |y - f|$ a.k.a. $\large L_1$ потеря или потеря Лапласа. На первый взгляд эта функция не кажется дифференцируемой, но на самом деле она определяет условную медиану. Медиана, как мы знаем, устойчива к выбросам, поэтому в некоторых случаях эта функция потерь лучше. Штраф за большие вариации не так тяжел, как это происходит в $\large L_2$.
- $ \large \begin{equation}  L(y, f) =\left\{   \begin{array}{@{}ll@{}}     (1 - \alpha) \cdot |y - f|, & \text{if}\ y-f \leq 0 \\     \alpha \cdot |y - f|, & \text{if}\ y-f >0  \end{array}\right. \end{equation}, \alpha \in (0,1)
$ a.k.a. $\large L_q$ потеря или потеря Квантиля. Вместо медианы он использует квантили. Например, $\large \alpha = 0.75$ соответствует 75%-ному квантилю. Мы можем видеть, что эта функция является асимметричной и штрафует наблюдения, которые находятся на правой стороне определенного квантиля.

<img src='https://habrastorage.org/web/6d5/e3a/09c/6d5e3a09c703491b947fde851e412ac0.png' width=60%>

Давайте используем функцию потерь $\large L_q$ на наших данных. Цель состоит в том, чтобы восстановить условный 75%-ный квантиль Косинуса. Давайте применим для GBM:
- Игрушечные данные $\large \left\{ (x_i, y_i) \right\}_{i=1, \ldots,300}$ ✓
- Число итераций $\large M = 3$ ✓;
- Функция потерь для квантилей $ \large \begin{equation}   L_{0.75}(y, f) =\left\{
\begin{array}{@{}ll@{}}    0.25 \cdot |y - f|, & \text{if}\ y-f \leq 0 \\     0.75 \cdot |y - f|, & \text{if}\ y-f >0   \end{array}\right. \end{equation} $ ✓;
- Градиент $\large L_{0.75}(y, f)$ - функция взвешенная по $\large \alpha = 0.75$. Мы собираемся обучить древовидную модель классификации:
$\large r_{i} = -\left[\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\right]_{f(x)=\hat{f}(x)} = $
$\large = \alpha I(y_i > \hat{f}(x_i) ) - (1 - \alpha)I(y_i \leq \hat{f}(x_i) ), \quad \mbox{for } i=1,\ldots,300$ ✓;
- Дерево решений как базовый алгоритм $\large h(x)$ ✓;
- Гиперпараметр деревьев: depth =  2 ✓;

Для нашего начального приближения мы возьмем необходимый квантиль из $\large y$. Однако мы ничего не знаем об оптимальных коэффициентах $\large \rho_t$, поэтому мы будем использовать стандартный поиск строк. Результаты таковы:

<img src='https://habrastorage.org/web/0e6/7dd/614/0e67dd614076499e91c8c4238457ae4d.png'   align='center'>

Мы можем наблюдать это на каждой итерации, $\large r_{i} $ возьмите только 2 возможных значения, но GBM все еще может восстановить нашу начальную функцию.

Общие результаты GBM с квантильной функцией потерь такие же, как и результаты с квадратичной функцией потерь, смещенной на$\large \approx 0.135$. Но если бы мы использовали 90%-ный квантиль, у нас не было бы достаточно данных из-за того, что классы стали бы несбалансированными. Мы должны помнить об этом, когда имеем дело с нестандартными задачами.

*Несколько слов о регрессионных функциях потерь*

Для регрессионных задач было разработано множество функций потерь, некоторые из которых имеют дополнительные свойства. Например, они могут быть надежными, как в [функции потерь Хьюбера](https://en.wikipedia.org/wiki/Huber_loss). Для небольшого числа выбросов функции потерь работают как $\large L_2$, но после определенного порога функция меняется на$\large L_1$. Это позволяет уменьшить влияние выбросов и сосредоточиться на общей картине.

Мы можем проиллюстрировать это на следующем примере. Данные генерируются из функции  $\large y = \frac{sin(x)}{x}$ с добавлением шума, смесь из нормального и Бернулли распределений. Мы показываем функции на графах A-D и соответствующий GBM на F-H (график E представляет начальную функцию):

<img src='https://habrastorage.org/web/130/05b/222/13005b222e8a4eb68c3936216c05e276.jpg'   align='center'> [Original size](https://habrastorage.org/web/130/05b/222/13005b222e8a4eb68c3936216c05e276.jpg).


В этом примере мы использовали сплайны в качестве базового алгоритма. Видите ли, это не всегда должны быть деревья для бустинга?

Мы ясно видим разницу между этими функциями потерь $\large L_2$, $\large L_1$, и Хьюбера. Если мы выберем оптимальные параметры для потери Хьюбера, то сможем получить наилучшую возможную аппроксимацию среди всех наших вариантов. Разницу можно увидеть также в 10%, 50% и 90%-квантилях.

К сожалению, функция потери Хьюбера поддерживается только очень немногими популярными библиотеками/пакетами; h2o поддерживает ее, а XGBoost - нет. Это относится и к другим вещам, которые являются более экзотическими, например [conditional expectiles](https://www.slideshare.net/charthur/quantile-and-expectile-regression), но это все равно может быть интересным знанием.

### Классификация функции потерь

Теперь давайте рассмотрим проблему бинарной классификации $\large y \in \left\{-1, 1\right\}$. Мы видели, что GBM может даже оптимизировать недифференцируемые функции потерь. Технически эту проблему можно решить с помощью регрессии $\large L_2$ потеря, но это было бы неправильно.

Распределение целевой переменной требует от нас использования логарифмического подобия, поэтому нам нужно иметь различные функции потерь для целей, умноженные на их предсказания:  $\large y \cdot f$. Наиболее распространенными вариантами были бы следующие:

- $\large L(y, f) = log(1 + exp(-2yf))$ a.k.a. Логистическая потеря или потеря Бернулли. Это имеет интересное свойство, которое наказывает даже правильно предсказанные классы, что помогает не только оптимизировать потери, но и еще больше отдалить классы друг от друга, даже если все классы предсказаны правильно.
- $\large L(y, f) = exp(-yf)$ a.k.a. Потеря AdaBoost. Классический AdaBoost эквивалентен GBM с этой функцией потерь. Концептуально эта функция очень похожа на логистические потери, но она имеет большой экспоненциальный штраф, если прогноз неверен.

<img src='https://habrastorage.org/web/bf5/9de/dcf/bf59dedcfd9d49b18e89ce342b09ce69.png' width=60%>

Давайте сгенерируем несколько новых игрушечных данных для нашей задачи классификации. За основу мы возьмем наш зашумленный Косинус и будем использовать знаковую функцию для классов целевой переменной. Наши игрушечные данные выглядят следующим образом (джиттер-шум добавляется для ясности):

<img src='https://habrastorage.org/web/e72/513/78b/e7251378bf6d459ab1aeea7a1f1996a1.jpg'>


Мы будем использовать логистические потери, чтобы найти то, что мы действительно увеличиваем. Итак, опять же, мы собрали то, что мы будем использовать для GBM:
- Игрушечные данные $\large \left\{ (x_i, y_i) \right\}_{i=1, \ldots,300}, y_i \in \left\{-1, 1\right\}$ ✓
- Число итераций $\large M = 3$ ✓;
- Логистические потери как функция потерь, ее градиент вычисляется следующим образом:
$\large r_{i} = \frac{2 \cdot y_i}{1 + exp(2 \cdot y_i \cdot \hat{f}(x_i)) }, \quad \mbox{for } i=1,\ldots,300$ ✓;
- Деревья решений как базовые алгоритмы $\large h(x)$ ✓;
- Используются для отбора признаков дерева решений: глубина дерева равна 2 ✓;

На этот раз инициализация алгоритма немного сложнее. Во-первых, наши классы несбалансированы (63% против 37%). Во-вторых, не существует известной аналитической формулы для инициализации нашей функции потерь, поэтому мы должны искать $\large \hat{f_0} = \gamma$ через поиск:

<img src='https://habrastorage.org/web/f8a/054/702/f8a05470271448d9bc0d4dc3e524a571.png' width=60%>


Наше оптимальное начальное приближение составляет около -0.273. Вы могли бы догадаться, что он был отрицательным, потому что выгоднее предсказать все, как самый популярный класс, но нет формулы для точного значения. А теперь давайте наконец запустим GBM и посмотрим, что на самом деле происходит под капотом:

<img src='https://habrastorage.org/web/7b4/ab0/5fa/7b4ab05fa0a543bfad94950e47f91568.png'   align='center'>

Алгоритм успешно восстановил разделение между нашими классами. Вы можете видеть, как разделяются "нижние" области, потому что деревья более уверены в правильном предсказании отрицательного класса и как формируются две ступени смешанных классов. Понятно, что у нас есть много правильно классифицированных наблюдений и некоторое количество наблюдений с большими ошибками, которые появились из-за шума в данных.

### Веса

Иногда возникает ситуация, когда мы хотим получить более конкретную функцию потерь для нашей проблемы. Например, в финансовых временных рядах мы можем захотеть придать больший вес большим движениям во временном ряду; для прогнозирования оттока более полезно предсказать отток клиентов с высоким LTV (или пожизненной ценностью: сколько денег клиент принесет в будущем).

<img src='https://habrastorage.org/web/0c0/ad0/3a4/0c0ad03a4c4b46bfa5bcd5101678c9c4.jpg'   align='center'>

Статистический воин изобрел бы свою собственную функцию потерь, выписал бы градиент для нее (для более эффективного обучения включите Гессиан) и тщательно проверил, удовлетворяет ли эта функция требуемым свойствам. Однако существует высокая вероятность ошибиться где-то, столкнуться с вычислительными трудностями и потратить чрезмерное количество времени на исследования.

Вместо этого был изобретен очень простой инструмент (который редко вспоминается на практике): взвешивание наблюдений и присвоение весовых функций. Простейшим примером такого взвешивания является установка весов для классового баланса. В общем случае, если мы знаем, что некоторое подмножество данных, как во входных переменных $\large x$ и в целевой переменной $\large y$, имеет большее значение для нашей модели, тогда мы просто придаем им больший вес $\large w(x,y)$. Основная цель состоит в том, чтобы выполнить общие требования к весам:
$$ \large w_i \in \mathbb{R}, \\
\large w_i \geq 0 \quad \mbox{for } i=1,\ldots,n, \\
\large \sum_{i = 1}^n w_i > 0 $$

Весовые коэффициенты позволяют значительно сократить время, затрачиваемое на корректировку функции потерь для решаемой задачи, а также стимулируют эксперименты со свойствами целевых моделей. Назначение этих весов полностью зависит от креативности. Мы просто добавляем скалярные веса:
$$ \large L_{w}(y,f) = w \cdot L(y,f), \\
\large r_{it} =   - w_i \cdot \left[\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\right]_{f(x)=\hat{f}(x)}, \quad \mbox{for } i=1,\ldots,n$$

Очевидно, что для произвольных весов мы не знаем статистических свойств нашей модели. Часто связывание весов со значениями $\large y$ это может быть слишком сложно. Например, использование весов, пропорциональных $\large |y|$ в $\large L_1$ функция потерь не является эквивалентной $\large L_2$ потери потому что градиент не будет учитывать значения самих прогнозов: $\large \hat{f}(x)$.

Мы упоминаем все это, чтобы лучше понять наши возможности. Давайте создадим несколько очень экзотических весов для наших игрушечных данных. Мы определим сильно асимметричную весовую функцию следующим образом:
$$ \large \begin{equation} w(x) =\left\{   \begin{array}{@{}ll@{}}     0.1, & \text{if}\ x \leq 0 \\     0.1 + |cos(x)|, & \text{if}\ x >0 \end{array}\right. \end{equation} $$

<img src='https://habrastorage.org/web/8c2/1b1/aa4/8c21b1aa47134f7aa46b15ef910369b2.png'   align='center'>

С помощью этих весов мы ожидаем получить два свойства: меньшую детализацию для отрицательных значений $\large x$ и форму функции, аналогичную исходному Косинусу. Мы берем другие настройки GBM из нашего предыдущего примера с классификацией, включая линейный поиск оптимальных коэффициентов. Давайте посмотрим, что у нас есть:

<img src='https://habrastorage.org/web/afc/cca/72a/afccca72a0774990b685de37b0fe9d9f.png'   align='center'>

Мы добились того результата, на который рассчитывали. Во-первых, мы можем видеть, насколько сильно отличаются псевдо-невязки; на начальной итерации они выглядят почти как исходный Косинус. Во-вторых, левая часть графика функции часто игнорировалась в пользу правой, которая имела больший вес. В-третьих, функция, которую мы получили на третьей итерации, получила достаточно внимания и стала выглядеть похожей на исходный Косинус (также начала немного перестраиваться).

Весы - это мощный, но рискованный инструмент, который мы можем использовать для управления свойствами нашей модели. Если вы хотите оптимизировать свою функцию потерь, то сначала стоит попытаться решить более простую задачу, но добавить веса к наблюдениям по своему усмотрению.

# 4. Заключение

Сегодня мы изучили теорию градиентного бустинга. GBM - это не просто какой-то конкретный алгоритм, а общая методология построения ансамблей моделей. Кроме того, эта методология достаточно гибка и расширяема-можно обучить большое количество моделей, учитывая различные функции потерь с различными весовыми функциями.

Практика и конкурсы мл показывают, что в стандартных задачах (за исключением изображений, аудио и очень разреженных данных) ГБМ часто является наиболее эффективным алгоритмом (не говоря уже о штабелировании и ансамблях высокого уровня, где ГБМ почти всегда является их частью). Кроме того, существует множество адаптаций GBM [для подкрепления обучения](https://arxiv.org/abs/1603.04119) (Minecraft, ICML 2016). Кстати, алгоритм Виолы-Джонса, который до сих пор используется в компьютерном зрении, [is based on AdaBoost](https://en.wikipedia.org/wiki/Viola%E2%80%93Jones_object_detection_framework#Learning_algorithm).

В этой статье мы намеренно опустили вопросы, касающиеся регуляризации GBM, стохастичности и гиперпараметров. Не случайно мы использовали небольшое количество итераций $\large M = 3$ на всем протяжении. Если бы мы использовали 30 деревьев вместо 3 и обучили GBM, как описано, результат не был бы таким предсказуемым:

<img src='../../img/topic10_good_fit.png'   align='center' width=60%>
<img src='../../img/topic10_overfitting.png'   align='center' width=60%>


<img src='https://habrastorage.org/web/27f/0f5/3be/27f0f53be9424cb1afaffb9a0e32909f.jpg'   align='center'>

[Interactive demo](http://arogozhnikov.github.io/2016/07/05/gradient_boosting_playground.html)